<a href="https://colab.research.google.com/github/Luzinsan/OPD_neuro_practice2/blob/main/Handwritten_Chinese_classes32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
import PIL.Image as Image
import matplotlib.pylab as plt
import tensorflow as tf
import os
import zipfile
import shutil

!git clone https://github.com/AI-FREE-Team/Traditional-Chinese-Handwriting-Dataset.git
OutputFolder = '/content/Handwritten_Data'
!rm -rf '/content/Handwritten_Data'

In [ ]:
SIZE = 32 # рассматриваем 32 класса

In [43]:
CompressedFiles = []

os.chdir('/content/Traditional-Chinese-Handwriting-Dataset/data')

for item in os.listdir():  
  if item.endswith('.zip'): # Check for ".zip" extension.
    file_path = os.path.abspath(item) # Get full path of the compressed file. 
    CompressedFiles.append(file_path)

for file in CompressedFiles:     
  # Construct a ZipFile object with the filename, and then extract it.
  zip_ref = zipfile.ZipFile(file).extractall(OutputFolder) 
  
  source_path = OutputFolder + '/cleaned_data(50_50)'
  img_list = os.listdir(source_path)

  for img in img_list:
      shutil.move(source_path + '/' + img, OutputFolder) # Move a file to another location. 
  
  shutil.rmtree(OutputFolder + '/cleaned_data(50_50)') 
  #print(f'Decompress successfully {file} ......')
  #print( 'Moving images according to traditional Chinese characters......' )

ImageList = os.listdir(OutputFolder)
ImageList = [img for img in ImageList if len(img)>1]
WordList = list(set([w.split('_')[0] for w in ImageList]))[:SIZE]

for w in WordList:
  try:
    os.chdir(OutputFolder) # Change the current working directory to OutputPath.
    os.mkdir(w) # Create the new word folder in OutputPath.
    MoveList = [img for img in ImageList if w in img]
                
  except: 
    os.chdir(OutputFolder)
    MoveList = [img for img in ImageList if w in img ]
  
  finally:            
    for img in MoveList:
      old_path = OutputFolder + '/' + img
      new_path = OutputFolder + '/' + w + '/' + img
      shutil.move( old_path, new_path )

#print( 'Data Deployment completed.' )

Decompress successfully /content/Traditional-Chinese-Handwriting-Dataset/data/cleaned_data(50_50)-20200420T071507Z-001.zip ......
Moving images according to traditional Chinese characters......
Decompress successfully /content/Traditional-Chinese-Handwriting-Dataset/data/cleaned_data(50_50)-20200420T071507Z-003.zip ......
Moving images according to traditional Chinese characters......
Decompress successfully /content/Traditional-Chinese-Handwriting-Dataset/data/cleaned_data(50_50)-20200420T071507Z-004.zip ......
Moving images according to traditional Chinese characters......
Decompress successfully /content/Traditional-Chinese-Handwriting-Dataset/data/cleaned_data(50_50)-20200420T071507Z-002.zip ......
Moving images according to traditional Chinese characters......
Data Deployment completed.


In [44]:
a=0
b=0

for item in os.listdir(OutputFolder):
  if (os.path.isdir(item)):  
    a += 1
    for i in os.listdir(OutputFolder + '/' + item):
      b +=1

#print ('Всего: ' + str(a) + ' слов (папка) / Всего: ' + str(b) + ' образцов')
#print ('В среднем каждое слово содержит: ' + str (b / a) + ' образцов')

Всего: 32 слов (папка) / Всего: 1675 образцов
В среднем каждое слово содержит: 52.34375 образцов


In [45]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.1)
train_dataset = image_generator.flow_from_directory(str(OutputFolder), class_mode='sparse', batch_size=5, target_size=(50, 50), subset='training')
valid_dataset = image_generator.flow_from_directory(str(OutputFolder), class_mode='sparse', batch_size=5, target_size=(50, 50), subset='validation')

Found 1516 images belonging to 32 classes.
Found 159 images belonging to 32 classes.


In [46]:
for image_b, label_b in train_dataset:
  #print("Image batch shape: ", image_b.shape)
  #print("Label batch shape: ", label_b.shape)
  image_batch = image_b
  label_batch = label_b
  #print(label_batch)
  break

Image batch shape:  (5, 50, 50, 3)
Label batch shape:  (5,)
[14.  1. 29. 22.  8.]


In [47]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(filters=16,  kernel_size=3, activation='relu', padding= 'same' , input_shape=(50,50,3)),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2),
  tf.keras.layers.Conv2D(filters=32,  kernel_size=3, activation='relu', padding= 'same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2),
  tf.keras.layers.Conv2D(filters=64,  kernel_size=3, activation='relu', padding= 'same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2),
  tf.keras.layers.Conv2D(filters=128,  kernel_size=3, activation='relu', padding= 'same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(SIZE, activation='softmax')
])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 50, 50, 16)        448       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 25, 25, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 25, 25, 32)        4640      
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 12, 12, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 12, 12, 64)        18496     
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 6, 6, 64)        

In [48]:
model.compile(
  optimizer=tf.keras.optimizers.Adadelta(learning_rate = 0.9),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)
EPOCHS = 10
history = model.fit(train_dataset,
                    validation_data=valid_dataset,
                    epochs=EPOCHS)

Epoch 1/10
304/304 [==============================] - 10s 30ms/step - loss: 4.9897 - accuracy: 0.0336 - val_loss: 3.5021 - val_accuracy: 0.0189
Epoch 2/10
304/304 [==============================] - 7s 24ms/step - loss: 3.5064 - accuracy: 0.0323 - val_loss: 3.4621 - val_accuracy: 0.0377
Epoch 3/10
304/304 [==============================] - 7s 24ms/step - loss: 3.4759 - accuracy: 0.0402 - val_loss: 3.4185 - val_accuracy: 0.0503
Epoch 4/10
304/304 [==============================] - 7s 24ms/step - loss: 3.4190 - accuracy: 0.0699 - val_loss: 3.3229 - val_accuracy: 0.0943
Epoch 5/10
304/304 [==============================] - 7s 24ms/step - loss: 3.3169 - accuracy: 0.0838 - val_loss: 3.2197 - val_accuracy: 0.1069
Epoch 6/10
304/304 [==============================] - 7s 24ms/step - loss: 3.1562 - accuracy: 0.1187 - val_loss: 2.8928 - val_accuracy: 0.1635
Epoch 7/10
304/304 [==============================] - 7s 24ms/step - loss: 2.8610 - accuracy: 0.1992 - val_loss: 2.7375 - val_accuracy: 0.207

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adadelta(learning_rate = 0.001),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)
EPOCHS = 100
history = model.fit(train_dataset,
                    validation_data=valid_dataset,
                    epochs=EPOCHS)

Epoch 1/100
304/304 [==============================] - 9s 26ms/step - loss: 1.4834 - accuracy: 0.5389 - val_loss: 1.6500 - val_accuracy: 0.5094
Epoch 2/100
304/304 [==============================] - 8s 26ms/step - loss: 1.3689 - accuracy: 0.5679 - val_loss: 1.6114 - val_accuracy: 0.5157
Epoch 3/100
304/304 [==============================] - 8s 25ms/step - loss: 1.3570 - accuracy: 0.5706 - val_loss: 1.5873 - val_accuracy: 0.5220
Epoch 4/100
304/304 [==============================] - 8s 26ms/step - loss: 1.2977 - accuracy: 0.5956 - val_loss: 1.5694 - val_accuracy: 0.5283
Epoch 5/100
304/304 [==============================] - 8s 25ms/step - loss: 1.3127 - accuracy: 0.5930 - val_loss: 1.5561 - val_accuracy: 0.5283
Epoch 6/100
304/304 [==============================] - 8s 25ms/step - loss: 1.2924 - accuracy: 0.5937 - val_loss: 1.5437 - val_accuracy: 0.5409
Epoch 7/100
304/304 [==============================] - 8s 25ms/step - loss: 1.2938 - accuracy: 0.5891 - val_loss: 1.5335 - val_accuracy:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([min(plt.ylim()),max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()